# Install pyodbc

https://docs.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver15#installing-microsoft-odbc-driver-for-sql-server

https://www.sqlshack.com/performing-crud-operations-with-a-python-sql-library-for-sql-server/

https://stackoverflow.com/questions/33725862/connecting-to-microsoft-sql-server-using-python/33787509

In [8]:
import pyodbc
server = 'officelaptop'
database = 'DBA'
username = 'grafana' 
password = 'grafana' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+'; TrustServerCertificate=yes;')
cursor = cnxn.cursor()

In [ ]:
# Sample Query
cursor.execute("select param_key, param_value from dbo.sma_params") 
row = cursor.fetchone() 
while row: 
    print(row)
    row = cursor.fetchone()

In [19]:
sum = lambda x,y: x+y

print(sum(10,5))

15


In [27]:
# Sample Query
cursor.execute("select top 3 param_key, param_value from dbo.sma_params") 
query_resultset = cursor.fetchall() 
col_names = [column[0] for column in query_resultset[0].cursor_description]
#print(col_names)

#col_index = lambda col: col_names.index(col)
#col_value = lambda row, col: row[col_index(col)]
col_value = lambda row, col: row[(lambda col: col_names.index(col))(col)]

for row in query_resultset:
    param_key = col_value(row,'param_key')
    param_value = col_value(row,'param_value')
    print(f"{param_key} => {param_value}")

alert_sender_email => sqlagentservice@gmail.com
credential_manager_database => DBA
cto_email_id => sqlagentservice@gmail.com


In [ ]:
# Sample Query
cursor.execute("select param_key, param_value from dbo.sma_params") 
result = cursor.fetchall() 
for row in result:
    #print(row)
    pass

In [ ]:
from prettytable import PrettyTable

pt_results = PrettyTable()
pt_results.field_names = [column[0] for column in result[0].cursor_description]
pt_results.add_rows(result)

print(pt_results)


In [ ]:
import pandas as pd
from pandas import DataFrame

df_columns = [column[0] for column in result[0].cursor_description]
df_results = pd.DataFrame.from_records(result,columns=df_columns)

print(df_results)

In [ ]:
#print(result)
#[row.param_key for row in result]

index_col = 'param_key'
columns = [column[0] for column in result[0].cursor_description]
indexOfIndexCol = columns.index(index_col)
#print(indexOfIndexCol)

#result[indexOfIndexCol]
indexValues = [row[indexOfIndexCol] for row in result]
print(indexValues)

for row in result:
    #print(row[indexOfIndexCol])
    pass


#[row.param_key for row in result]


**Connection String with APP name**

In [ ]:
import pyodbc
cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost;'
                      'Database=StackOverflow;'
                      'Trusted_Connection=yes;'
                      'APP=sql-server-load-generator.py')
cursor = cnxn.cursor()

# Sample Query
cursor.execute("SELECT @@version; WAITFOR DELAY '00:20:00'") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()